# Dependencies

In [17]:
import pandas
from keys import census_key
import requests
from sqlalchemy import create_engine

# Extract

Read in population data

In [18]:
population_data = pandas.read_csv('population_data.csv')

Read in diversity data

In [19]:
diversity_data = pandas.read_csv('diversityindex.csv')

# Transform

Filter out unnecessary columns

In [20]:
new_pop_data = population_data[['GEO.display-label','respop72013']]

Clean county names and create a new column of names

In [21]:
county_list = []
for county in new_pop_data['GEO.display-label']:
    try:
        county_list.append(county.split(', ')[0])
    except:
        county_list.append(county)
new_pop_data['County Name'] = county_list

C:\Users\Jake\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


Create a list that contains the state and county part of each location

In [22]:
state_lst = []
county_lst2 = []
for row in diversity_data['Location']:
    try:
        state_lst.append(row.split(', ')[1])
        county_lst2.append(row.split(', ')[0])
    except:
        state_lst.append(row)
        county_lst2.append(row)

Use the newly created list to add a state column to the data table

In [23]:
diversity_data['State'] = state_lst
diversity_data['County Name'] = county_lst2

Filter diversity data to only include data from California

In [24]:
ca_diversity = diversity_data.loc[diversity_data['State']=='CA']

combine tables on the county name

In [25]:
combined_df = pandas.merge(ca_diversity, new_pop_data, on = 'County Name', how = 'left')

Filter out unnecessary columns

In [26]:
combined_df = combined_df[['County Name', 'Black or African American alone, percent, 2013', 'American Indian and Alaska Native alone, percent, 2013', 'Asian alone, percent, 2013','Native Hawaiian and Other Pacific Islander alone, percent,', 'Two or More Races, percent, 2013', 'Hispanic or Latino, percent, 2013', 'White alone, not Hispanic or Latino, percent, 2013', 'respop72013']]

Rename the columns to be more useful

In [27]:
combined_df = combined_df.rename(columns = {'Black or African American alone, percent, 2013': '% Black or African American', 'American Indian and Alaska Native alone, percent, 2013':'% Native American or Alaska Native', 'Asian alone, percent, 2013':'% Asian', 'Native Hawaiian and Other Pacific Islander alone, percent,': '% Pacific Islander', 'Two or More Races, percent, 2013': '% Two or more', 'Hispanic or Latino, percent, 2013': '% Hispanic or Latino', 'White alone, not Hispanic or Latino, percent, 2013': '% White', 'respop72013':'Population' })

Convert population data type from string to float

In [28]:
combined_df['Population'] = combined_df['Population'].astype(float)

Add in new columns to convert all percentages to numbers

In [29]:
combined_df['# Black or African American'] = round(combined_df['% Black or African American']/100*combined_df['Population'],0)
combined_df['# Native American or Alaska Native'] = round(combined_df['% Native American or Alaska Native']/100*combined_df['Population'],0)
combined_df['# of Asian'] = round(combined_df['% Asian']/100*combined_df['Population'], 0)
combined_df['# of Pacific Islander'] = round(combined_df['% Pacific Islander']/100*combined_df['Population'], 0)
combined_df['# of Two or More'] = round(combined_df['% Two or more']/100*combined_df['Population'], 0)
combined_df['# of Hispanic or Latino'] = round(combined_df['% Hispanic or Latino']/100*combined_df['Population'], 0)
combined_df['# of White'] = round(combined_df['% White']/100*combined_df['Population'], 0)

Filter out unnecessary columns

In [30]:
combined_df = combined_df[['County Name','# Black or African American', '# Native American or Alaska Native', '# of Asian', '# of Pacific Islander', '# of Two or More', '# of Hispanic or Latino', '# of White']]
combined_df.head()

,County Name,# Black or African American,# Native American or Alaska Native,# of Asian,# of Pacific Islander,# of Two or More,# of Hispanic or Latino,# of White
0,Alameda County,195782.0,18947.0,445247.0,15789.0,82102.0,358408.0,524192.0
1,Solano County,63293.0,5522.0,65417.0,4248.0,28461.0,107047.0,169915.0
2,Santa Clara County,53999.0,26069.0,634956.0,9310.0,74482.0,499027.0,631232.0
3,San Mateo County,22421.0,6726.0,201043.0,11958.0,32884.0,189833.0,307170.0
4,Sacramento County,157910.0,23394.0,225168.0,17546.0,86266.0,326055.0,691588.0


# Load

In [31]:
engine = create_engine('sqlite://', echo=False)

In [32]:
combined_df.to_sql('ethnicity_data', con=engine)

In [35]:
engine.execute("SELECT * FROM ethnicity_data").fetchall()

[(0, 'Alameda County', 195782.0, 18947.0, 445247.0, 15789.0, 82102.0, 358408.0, 524192.0),
 (1, 'Solano County', 63293.0, 5522.0, 65417.0, 4248.0, 28461.0, 107047.0, 169915.0),
 (2, 'Santa Clara County', 53999.0, 26069.0, 634956.0, 9310.0, 74482.0, 499027.0, 631232.0),
 (3, 'San Mateo County', 22421.0, 6726.0, 201043.0, 11958.0, 32884.0, 189833.0, 307170.0),
 (4, 'Sacramento County', 157910.0, 23394.0, 225168.0, 17546.0, 86266.0, 326055.0, 691588.0),
 (5, 'Contra Costa County', 105044.0, 10942.0, 173979.0, 6565.0, 54710.0, 272457.0, 506617.0),
 (6, 'San Joaquin County', 57055.0, 14088.0, 110588.0, 4931.0, 35923.0, 282456.0, 243715.0),
 (7, 'San Francisco County', 50247.0, 6700.0, 288080.0, 4187.0, 34335.0, 128129.0, 348376.0),
 (8, 'Orange County', 65402.0, 34258.0, 597958.0, 12457.0, 99660.0, 1065112.0, 1326719.0),
 (9, 'Los Angeles County', 921570.0, 150256.0, 1462492.0, 40068.0, 290495.0, 4838244.0, 2724642.0),
 (10, 'San Diego County', 179830.0, 41746.0, 375716.0, 19268.0, 134873.0